In [4]:
institutions_list = ['World Health Organization', 'WHO', 'Research', 'Reproductive', 'Study', 'Health', 
             'GROUP', 'NETWORK', 'Team', 'University', 'Project', 'IEDEA', 
             'Consortium', 'committee', 'all the authors', 'for IeDEA-Southern Africa', 'Systematic', 
             'collaborations', 'Organizacion', 'College', 'Consortium', 'Association', 'Survey', 'Expert',
             'de la Salud', 'Control','collaborations', 'Collaboration', 'committee', 'Universite', 'BURKINA FASO']
institutions_list = [institutions.upper() for institutions in institutions_list]


def prepare_std_name(authors):
    last_name = []
    first_letter_name = []
    standard_name = []
    for nome in authors['full_name']:
        nome = re.sub(' +', ' ', unidecode(nome).replace('-', ' ').strip().upper())
        last_name += [nome.split(" ")[-1]]
        first_letter_name += [nome.split(" ")[0][0]]
        standard_name += [nome.split(" ")[0][0] + " " + nome.split(" ")[-1]]
    
    authors['last_name'] = last_name
    authors['first_letter_name'] = first_letter_name
    authors['standard_name'] = standard_name
    
    return authors

def get_longest_name(group):        
    names_list = group['unique'].iloc[0].tolist()
    candidate_name = unidecode(max(names_list, key=len))

    while len(candidate_name.split(' ')) == 1 and len(names_list) > 1:
        names_list.remove(candidate_name)
        candidate_name = unidecode(max(names_list, key=len))   
    return candidate_name


def deduplica_nome(inicial_data, column_group, column_agg):
    return_data = pd.DataFrame(inicial_data)
    names_variation_unique = inicial_data.groupby(column_group)[column_agg].agg(['unique']).reset_index()
    #authors.to_csv(path+'names_variation_unique.csv', encoding="utf-8", index=False)

    return_data = return_data.reset_index()
    for index, row in return_data.iterrows():
        group = names_variation_unique[names_variation_unique[column_group] == row[column_group]]
        candidate_name = get_longest_name(group)
        
        return_data.loc[index, 'deduplicated_name_std'] = candidate_name
        return_data.loc[index, 'names_variation_std'] = ", ".join(group.unique.iloc[0].tolist())
        return_data.loc[index, 'n_variacoes_std'] = len(group.unique.iloc[0].tolist())
        #print(row[column_group], candidate_name, ", ".join(group.unique.iloc[0].tolist()), len(group.unique.iloc[0].tolist()))
            
    return return_data

def gess_gender_author(name):
    first_name = name.split(' ')[0]
    guessed_gender = 'unknown'
    
    if len(first_name) > 1:
        guessed_gender = d.get_gender(first_name.title())
    
    if guessed_gender =='mostly_female':
        guessed_gender = 'female'
    if guessed_gender =='mostly_male':
        guessed_gender = 'male'
    
    return guessed_gender


def create_deduplicated_columns(authors):
    deduplicate_data =  deduplica_nome(authors, "standard_name", "full_name")    
    return deduplicate_data


def is_groups_name(nome):
    if any(s.upper() in nome.upper() for s in institutions_list):
        return True
    else:
        return False

def remove_groups_names(authors):  
    institutions = []
    for nome in authors.full_name.value_counts().index:
        if is_groups_name(nome):
            authors = authors[authors.full_name != nome]
            if nome not in institutions:
                institutions += [nome]
                
    return authors, institutions


def get_list_names(authors, author):
    list_names = authors[authors['deduplicated_name_std'] == author].iloc[0]['names_variation_std'].split(", ")    
    return list_names

def guess_gender_in_list(list_names, last=False):
    #print(list_names)
    author_guessed_gender = 'unknown'
    index_name = 0
    while (author_guessed_gender == 'unknown' and len(list_names) > index_name):
        if last:
            name = list_names[index_name].split(" ")[-1]
        else:
            name = list_names[index_name].split(" ")[0]
            
        author_guessed_gender = gess_gender_author(name)
        #print(name, author_guessed_gender, index_name)
        index_name = index_name + 1
    
    return author_guessed_gender

def remap_affiliation(authors):   
    secondary_list = []
    authors['countries'] = [[] for i in authors['affiliation']]  
    authors['countries'] = authors['countries'].astype('object')
    affiliations_unique = pd.read_csv(path+'..\\Reviewed Files\\01_affiliations_unique_with_countries_v5.csv')
    affiliations_unique.original = [clean_string(affiliation, True) for affiliation in affiliations_unique.original]
    with alive_bar(len(authors['affiliation']), force_tty=True) as bar:
        for index, row in authors.iterrows():
            affiliation_list= row['affiliation']
            remaped_list = []
            country_list = []
            if affiliation_list != "Collaborators":  
                #print(affiliation_list)
                affiliations = affiliation_list
                
                for affiliation in (affiliations):
                    if not pd.isnull(affiliation):
                        remaps = affiliations_unique.loc[affiliations_unique.original == clean_string(affiliation, True)].reset_index()
                        if len(remaps) > 0:
                            remaped_list += [remaps.iloc[0]['remap']]
                            country_list += [remaps.iloc[0]['country']]
                        else:
                            print(affiliation)
                            remaped_list += [affiliation]
                            secondary_list += [clean_string(str(affiliation), True)]
                authors.at[index, 'affiliation'] = remaped_list
                authors.at[index, 'countries'] = country_list
            bar()
    df = pd.DataFrame(secondary_list, columns=["affiliations_secondary"])
    df.to_csv(path+'secondary_list.csv', index=False)

In [5]:
import csv
import json
import time
import pandas as pd
import numpy as np

import gender_guesser.detector as gender

import requests
import re
import jellyfish
import spacy
import geonamescache

from functions import *

from os import makedirs
from os.path import exists, join
from unidecode import unidecode
from collections import Counter
from pandas_profiling import ProfileReport
from spacy import displacy 
from alive_progress import alive_bar

#nlp = spacy.load("en_core_web_lg")
d = gender.Detector()
gc = geonamescache.GeonamesCache()
# gets nested dictionary for countries
countries = gc.get_countries()
# gets nested dictionary for cities
cities = gc.get_cities()

import sys  
sys.path.insert(0, 'D:\Google Drive\Analises\Codigos python')
from build_features import *


import warnings
warnings.filterwarnings('ignore')

In [6]:
path = 'C:\\Users\\livia\\Dropbox\\HRP Alliance authorship paper\\'

df_reports = pd.read_csv(path + "Data 2022-11-15\\Merged_database_encoding.csv", encoding="utf-8")
df_reports["Constant"] = ["X" for i in df_reports.Origin]
df_reports.Tags.value_counts(dropna=False)
tabelaEstiloArtigoCategoricas(df_reports, 'Constant', 'Origin', probabilidade='col')
#df_reports.columns

Constant                                        X              All          
distribuição                                    N         %      N         %
Origin                                                                      
HRP publications 1972-2022_24.03.2022        2335  (20.37%)   2335  (20.37%)
HRP-Vanessa-1133-06.04.2022                  1133   (9.88%)   1133   (9.88%)
HRPPublicationsDatabase_UpdateDecember2014   3960  (34.55%)   3960  (34.55%)
VB_remained_reports                           198   (1.73%)    198   (1.73%)
VB_reports_screened                           857   (7.48%)    857   (7.48%)
covidence-doi                                2979  (25.99%)   2979  (25.99%)
All                                         11462  (100.0%)  11462  (100.0%)

In [7]:
df_reports.sort_values(by="Tags", inplace=True)
print("Original number", len(df_reports))
df_reports = df_reports[~(df_reports['DOI'].duplicated()) | (df_reports['DOI'].isna())]
print("Removed duplicated doi", len(df_reports))
df_reports = df_reports[~(df_reports['PMID'].duplicated()) | df_reports['PMID'].isna()]
print("Removed duplicated pmid", len(df_reports))
df_reports = df_reports[~(df_reports['Title'].duplicated()) | df_reports['Title'].isna()]
print("Removed duplicated title", len(df_reports))
df_reports = df_reports[~(df_reports['Term'].duplicated()) | df_reports['Term'].isna()]
print("Removed duplicated term", len(df_reports))

tabelaEstiloArtigoCategoricas(df_reports, 'Constant', 'Origin', probabilidade='col')

Original number 11462
Removed duplicated doi 8154
Removed duplicated pmid 7909
Removed duplicated title 7232
Removed duplicated term 7012


Constant                                       X             All          
distribuição                                   N         %     N         %
Origin                                                                    
HRP publications 1972-2022_24.03.2022          4   (0.06%)     4   (0.06%)
HRPPublicationsDatabase_UpdateDecember2014  3380   (48.2%)  3380   (48.2%)
VB_remained_reports                           51   (0.73%)    51   (0.73%)
VB_reports_screened                          785   (11.2%)   785   (11.2%)
covidence-doi                               2792  (39.82%)  2792  (39.82%)
All                                         7012  (100.0%)  7012  (100.0%)

In [8]:
df_reports.Tags.value_counts(dropna=False)

Unverified        3384
Not included      2239
Included by VB    1389
Name: Tags, dtype: int64

In [9]:
tabelaEstiloArtigoCategoricas(df_reports, 'Tags', 'Origin', probabilidade='col')

Tags                                       Included by VB            \
distribuição                                            N         %   
Origin                                                                
HRP publications 1972-2022_24.03.2022                   0    (0.0%)   
HRPPublicationsDatabase_UpdateDecember2014              0    (0.0%)   
VB_remained_reports                                    37   (2.66%)   
VB_reports_screened                                   508  (36.57%)   
covidence-doi                                         844  (60.76%)   
All                                                  1389  (100.0%)   

Tags                                       Not included           Unverified  \
distribuição                                          N         %          N   
Origin                                                                         
HRP publications 1972-2022_24.03.2022                 0    (0.0%)          4   
HRPPublicationsDatabase_UpdateDecember2014            0    (0.0%)       3380   
VB_remained_reports                                  14   (0.63%)          0   
VB_reports_screened                                 277  (12.37%)          0   
covidence-doi                                      1948   (87.0%)          0   
All                                                2239  (100.0%)       3384   

Tags                                                   All            
distribuição                                       %     N         %  
Origin                                                                
HRP publications 1972-2022_24.03.2022        (0.12%)     4   (0.06%)  
HRPPublicationsDatabase_UpdateDecember2014  (99.88%)  3380   (48.2%)  
VB_remained_reports                           (0.0%)    51   (0.73%)  
VB_reports_screened                           (0.0%)   785   (11.2%)  
covidence-doi                                 (0.0%)  2792  (39.82%)  
All                                         (100.0%)  7012  (100.0%)

In [10]:
df_reports.Term.value_counts(dropna=False)

10.1186/s12978-016-0262-5                                                                                                                                                                                                                                                   1
30644242                                                                                                                                                                                                                                                                    1
9130653                                                                                                                                                                                                                                                                     1
12730477                                                                                                                                                                                      

In [11]:
database, complete_list_authors = collect_all_papers(list(df_reports.Term), serach_by_pmid = False)

database.to_csv(path + 'Data 2022-11-15\\papers_pubmed.csv', encoding='utf-8')
complete_list_authors.to_csv(path + 'Data 2022-11-15\\authors.csv', encoding='utf-8', index=False)

|████████████████████████████████████████| 7012/7012 [100%] in 2:39:51.5 (0.73/s)                                       


In [12]:
print("{} Total of searched rows (df_reports)".format(len(df_reports)))
print("{} Total of retrieved rows (database)".format(len(database)))

7012 Total of searched rows (df_reports)
5197 Total of retrieved rows (database)


In [13]:
papers = pd.merge(df_reports[["Term", "Title","Tags", 'Origin']], database, how="left", right_on="term", left_on="Term")
papers["retrieved"] = ["Yes" if not pd.isnull(item) else "Not Found" for item in papers.title]

papers['publication_year'] = pd.to_numeric(papers['publication_year'], errors='coerce')
print("{} Size of merged database (papers)".format(len(papers)))

7012 Size of merged database (papers)


In [14]:
papers.loc[papers.Tags.isin(["??", "Repeat from key: R8DNF7RQ", "Not found"]), "Tags"] = "Unverified"

In [15]:
papers.pmid.value_counts(dropna=False)

NaN         1815
35777803       5
22398174       3
23271957       3
3585867        3
            ... 
19533478       1
27473210       1
10487163       1
12179430       1
33432622       1
Name: pmid, Length: 4872, dtype: int64

In [16]:
papers = papers[~(papers['pmid'].isna())]
print("Original number", len(papers))

Original number 5197


In [17]:
papers.sort_values(by="Tags", inplace=True)
print("Original number", len(papers))
papers = papers[~(papers['doi'].duplicated()) | (papers['doi'].isna())]
print("Removed duplicated doi", len(papers))
papers = papers[~(papers['pmid'].duplicated()) | papers['pmid'].isna()]
print("Removed duplicated pmid", len(papers))
papers = papers[~(papers['title'].duplicated()) | papers['title'].isna()]
print("Removed duplicated title", len(papers))
papers = papers[~(papers['term'].duplicated()) | papers['term'].isna()]
print("Removed duplicated term", len(papers))

Original number 5197
Removed duplicated doi 4395
Removed duplicated pmid 4395
Removed duplicated title 4382
Removed duplicated term 4382


In [18]:
tabelaEstiloArtigoCategoricas(papers, 'Tags', 'retrieved', probabilidade='col')

Tags         Included by VB           Not included           Unverified  \
distribuição              N         %            N         %          N   
retrieved                                                                 
Yes                    1204  (100.0%)         1805  (100.0%)       1373   
All                    1204  (100.0%)         1805  (100.0%)       1373   

Tags                     All            
distribuição         %     N         %  
retrieved                               
Yes           (100.0%)  4382  (100.0%)  
All           (100.0%)  4382  (100.0%)

In [19]:
papers.term.value_counts(dropna=False)

10.1186/s12978-016-0262-5                                                                                        1
Plasma soluble HLA-G is a potential biomarker for diagnosis of colorectal gastric oesophageal and lung cancer    1
28541378                                                                                                         1
31891599                                                                                                         1
11861054                                                                                                         1
                                                                                                                ..
10796166                                                                                                         1
32285856                                                                                                         1
Views of Chinese parents on the provision of contraception to unmarried youth   

In [20]:
tabelaEstiloArtigoCategoricas(papers, 'Tags', 'Origin', probabilidade='col')

Tags                                       Included by VB            \
distribuição                                            N         %   
Origin                                                                
HRPPublicationsDatabase_UpdateDecember2014              0    (0.0%)   
VB_reports_screened                                   434  (36.05%)   
covidence-doi                                         770  (63.95%)   
All                                                  1204  (100.0%)   

Tags                                       Not included           Unverified  \
distribuição                                          N         %          N   
Origin                                                                         
HRPPublicationsDatabase_UpdateDecember2014            0    (0.0%)       1373   
VB_reports_screened                                 141   (7.81%)          0   
covidence-doi                                      1664  (92.19%)          0   
All                                                1805  (100.0%)       1373   

Tags                                                   All            
distribuição                                       %     N         %  
Origin                                                                
HRPPublicationsDatabase_UpdateDecember2014  (100.0%)  1373  (31.33%)  
VB_reports_screened                           (0.0%)   575  (13.12%)  
covidence-doi                                 (0.0%)  2434  (55.55%)  
All                                         (100.0%)  4382  (100.0%)

In [21]:
papers_to_check_authors = list(papers[(papers.Tags == "Unverified") & (papers.retrieved == "Yes")].pmid)
authors_to_check = complete_list_authors[complete_list_authors.pmid.isin(papers_to_check_authors)]

authors_to_check, institutions = remove_groups_names(authors_to_check)

In [22]:
authors_to_check = prepare_std_name(authors_to_check)
authors_to_check.tail(2)

,full_name,affiliation,index_authorship,pmid,last_name,first_letter_name,standard_name
5,EMI SUZUKI,[],6,23584466,SUZUKI,E,E SUZUKI
6,JOHN WILMOTH,[],7,23584466,WILMOTH,J,J WILMOTH


In [23]:
authors_to_check = create_deduplicated_columns(authors_to_check)
authors_to_check.head(2)

,index,full_name,affiliation,index_authorship,pmid,last_name,first_letter_name,standard_name,deduplicated_name_std,names_variation_std,n_variacoes_std
0,0,Q FENG,[DEPARTMENT OF PRE CLINICAL SCIENCES LEICESTER...,1,11170823,FENG,Q,Q FENG,QIANG FENG,"Q FENG, QING FENG, QIANG FENG",3.0
1,1,K LIU,[],2,11170823,LIU,K,K LIU,K L LIU,"K LIU, K L LIU, KUI LIU",3.0


In [24]:
hrp_authors_deduplicated_name = pd.read_csv(path+"Reviewed Files\\hrp_authors_deduplicated_name.csv")
hrp_authors_deduplicated_name.head(2)

,Unnamed: 0,deduplicated_name_std
0,0,ABIODUN ADANIKIN
1,1,ADRIANO BUENO TAVARES


In [25]:
##get papers with hrp authors
columns_name = ['index_authorship', 'pmid', 'deduplicated_name_std']
list_papers_hrp_authors = pd.DataFrame(columns = columns_name)
for name in hrp_authors_deduplicated_name.deduplicated_name_std:
    candidates = authors_to_check[authors_to_check.deduplicated_name_std == name][columns_name]
    if len(candidates) > 0:
        list_papers_hrp_authors = pd.concat([list_papers_hrp_authors, candidates])
    ##authors_to_check.deduplicated_name_std
    
list_papers_hrp_authors.drop_duplicates(inplace=True)
list_papers_hrp_authors.head(2)

list_papers_hrp_authors_gb = list_papers_hrp_authors.groupby("pmid").agg(deduplicated_name_std=("deduplicated_name_std",", ".join))

In [26]:
papers = pd.merge(papers, list_papers_hrp_authors_gb, how="left", right_on="pmid", left_on="pmid")

papers.at[~pd.isnull(papers.deduplicated_name_std), "Tags"] = "Has HRP authors"
papers.loc[(papers.retrieved == "Not Found") & (papers.Tags == "Unverified"), "Tags"] = "Not Found"

In [27]:
print("{}\tNumber of papers from 2022+".format(len(papers[papers.publication_year >= 2022].pmid)))
print("{}\tNumber of papers with type Published Erratum".format(len(papers[papers.publication_type == "Published Erratum"].pmid)))
print("{}\tNumber of papers with type Interview".format(len(papers[papers.publication_type == "Interview"].pmid)))

##pmid_to_delete = list(papers[papers.doi == "10.1159/000343054"].pmid) -> interview
pmid_to_delete = []
pmid_to_delete += [27227232]
pmid_to_delete += list(papers[papers.publication_type == "Published Erratum"].pmid)
pmid_to_delete += list(papers[papers.publication_type == "Interview"].pmid)
pmid_to_delete += list(papers[papers.publication_year >= 2022].pmid)

63	Number of papers from 2022+
17	Number of papers with type Published Erratum
1	Number of papers with type Interview


In [28]:
papers.loc[papers.publication_year >= 2022, "Tags"] = "Not included"
papers.loc[papers.publication_type == "Published Erratum", "Tags"] = "Not included"
papers.loc[papers.publication_type == "Interview", "Tags"] = "Not included"

In [29]:
tabelaEstiloArtigoCategoricas(papers, 'Tags', 'Origin', probabilidade='col')

Tags                                       Has HRP authors            \
distribuição                                             N         %   
Origin                                                                 
HRPPublicationsDatabase_UpdateDecember2014             242  (100.0%)   
VB_reports_screened                                      0    (0.0%)   
covidence-doi                                            0    (0.0%)   
All                                                    242  (100.0%)   

Tags                                       Included by VB            \
distribuição                                            N         %   
Origin                                                                
HRPPublicationsDatabase_UpdateDecember2014              0    (0.0%)   
VB_reports_screened                                   433  (37.36%)   
covidence-doi                                         726  (62.64%)   
All                                                  1159  (100.0%)   

Tags                                       Not included           Unverified  \
distribuição                                          N         %          N   
Origin                                                                         
HRPPublicationsDatabase_UpdateDecember2014            2   (0.11%)       1129   
VB_reports_screened                                 142   (7.67%)          0   
covidence-doi                                      1708  (92.22%)          0   
All                                                1852  (100.0%)       1129   

Tags                                                   All            
distribuição                                       %     N         %  
Origin                                                                
HRPPublicationsDatabase_UpdateDecember2014  (100.0%)  1373  (31.33%)  
VB_reports_screened                           (0.0%)   575  (13.12%)  
covidence-doi                                 (0.0%)  2434  (55.55%)  
All                                         (100.0%)  4382  (100.0%)

In [31]:
#papers.to_csv(path + "papers_completed.csv", index=False)
papers.to_csv(path + "Data 2022-11-15\\papers_completed_v1.csv", index=False)